### Generate Data --- PARITY

In [1]:
import os, random, json, re, math
import pandas as pd
import numpy as np
from tqdm import trange, tqdm
import numpy as np
from collections import Counter
from itertools import permutations,combinations
TASK = "parity"

In [64]:
# length_extrapolation
samples_per_num_digit = 1000
data = {}
for num_digit in trange(129, 257):
    strings = []
    while len(strings) < samples_per_num_digit:
        for num_ones in range(num_digit, -1, -1):
            s = "1"*num_ones + "0"*(num_digit-num_ones)
            while True:
                candidate = "".join(random.sample(s, len(s)))
                if candidate in strings and (num_ones-num_digit)*num_ones == 0: break
                if not candidate in strings:
                    strings.append(candidate)
                    break

    random.shuffle(strings)
    data[num_digit] = strings
json.dump(data, open(f"../data/finetune/{TASK}/finetune_129_256.json", "w"), indent=2)
            

  0%|          | 0/128 [00:00<?, ?it/s]

100%|██████████| 128/128 [00:08<00:00, 14.23it/s]


In [5]:
# uniform, uniform_hard, uniform_hard+ 1000 instances
samples_per_num_digit = 1000
data = {}
for num_digit in trange(10, 129):
    strings = []
    while len(strings) < samples_per_num_digit:
        for num_ones in range(num_digit, -1, -1):
            s = "1"*num_ones + "0"*(num_digit-num_ones)
            
            # special_care, for those choose(num_digit, num_ones) < ceil(samples_per_num_digit / num_digit)
            if min(num_digit-num_ones, num_ones) == 1 or num_digit + min(num_digit-num_ones, num_ones) <= 16:
                for positions_for_ones in combinations(range(num_digit), num_ones):
                    candidate = "0"*num_digit
                    for p in positions_for_ones: 
                        candidate = candidate[:p] + "1" + candidate[p+1:]
                    if not candidate in strings:
                        strings.append(candidate)
                        break
                continue
            while True:
                candidate = "".join(random.sample(s, len(s)))
                if candidate in strings and (num_ones-num_digit)*num_ones == 0: break
                if not candidate in strings:
                    strings.append(candidate)
                    break
    
    random.shuffle(strings)
    data[num_digit] = strings

json.dump(data, open(f"../data/finetune/{TASK}/finetune_1000instances.json", "w"), indent=2)
            


100%|██████████| 119/119 [00:05<00:00, 22.26it/s]


In [6]:
# uniform, uniform_hard, uniform_hard+ 2000 instances
samples_per_num_digit = 2000
data = {}
for num_digit in trange(11, 129):
    strings = []
    while len(strings) < samples_per_num_digit:
        for num_ones in range(num_digit, -1, -1):
            s = "1"*num_ones + "0"*(num_digit-num_ones)
            
            # special_care, for those choose(num_digit, num_ones) < ceil(samples_per_num_digit / num_digit)
            if min(num_digit-num_ones, num_ones) == 1 or num_digit + min(num_digit-num_ones, num_ones) <= 20:
                for positions_for_ones in combinations(range(num_digit), num_ones):
                    candidate = "0"*num_digit
                    for p in positions_for_ones: 
                        candidate = candidate[:p] + "1" + candidate[p+1:]
                    if not candidate in strings:
                        strings.append(candidate)
                        break
                continue
            while True:
                candidate = "".join(random.sample(s, len(s)))
                if candidate in strings and (num_ones-num_digit)*num_ones == 0: break
                if not candidate in strings:
                    strings.append(candidate)
                    break
    
    random.shuffle(strings)
    data[num_digit] = strings

json.dump(data, open(f"../data/finetune/{TASK}/finetune_2000instances.json", "w"), indent=2)
            


  0%|          | 0/118 [00:00<?, ?it/s]

100%|██████████| 118/118 [00:23<00:00,  5.09it/s]


In [10]:
strings = data[18]
Counter([s.count('1') for s in strings])

Counter({9: 131,
         16: 131,
         13: 131,
         3: 131,
         15: 131,
         5: 131,
         7: 131,
         11: 131,
         12: 131,
         10: 131,
         2: 131,
         1: 18,
         4: 131,
         14: 131,
         6: 131,
         8: 131,
         17: 18,
         18: 1,
         0: 1})

In [12]:
# Uniform split
samples_per_num_digit_train, samples_per_num_digit_val = 100, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune_1000instances.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

answers = []

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        answer = '+' if s.count('1')%2==0 else '-'
        answers.append(answer)
        row = {
            "input_str": [s],
            "answer": [answer]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        row = {
            "input_str": [s],
            "answer": ['+' if s.count('1')%2==0 else '-']
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

print("Check if answers are uniformly distributed across [+, -]")
print(Counter(answers).most_common(10))
train.to_csv(f"../data/finetune/{TASK}/uniform_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/uniform_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 119/119 [00:07<00:00, 16.79it/s]


11900 11900
Check if answers are uniformly distributed across [+, -]
[('+', 5967), ('-', 5933)]


In [65]:
# length_extrapolation split
samples_per_num_digit_train, samples_per_num_digit_val = 0, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune_129_256.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        row = {
            "input_str": [s],
            "answer": ['+' if s.count('1')%2==0 else '-']
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        row = {
            "input_str": [s],
            "answer": ['+' if s.count('1')%2==0 else '-']
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

val.to_csv(f"../data/finetune/{TASK}/length_extrapolation/val.csv", index=False)

samples_per_num_digit = 1026


  0%|          | 0/128 [00:00<?, ?it/s]

100%|██████████| 128/128 [00:03<00:00, 36.65it/s]

0 12800


In [4]:
# Random Labels
samples_per_num_digit_train, samples_per_num_digit_val = 100, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune_1000instances.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

answers = []

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        answer = random.choice(['+', '-'])
        answers.append(answer)
        row = {
            "input_str": [s],
            "answer": [answer]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        row = {
            "input_str": [s],
            "answer": [random.choice(['+', '-'])]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

print("Check if answers are uniformly distributed across [+, -]")
print(Counter(answers).most_common(10))
train.to_csv(f"../data/finetune/{TASK}/random_labels_large/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/random_labels_large/val.csv", index=False)


samples_per_num_digit = 2000


  0%|          | 0/118 [00:00<?, ?it/s]

100%|██████████| 118/118 [01:32<00:00,  1.28it/s]


118000 11800
Check if answers are uniformly distributed across [+, -]
[('-', 59298), ('+', 58702)]
